In [5]:
import random
import copy
from matplotlib import pyplot as plt
from pandas import *
from operator import itemgetter
import numpy as np
import time

start_time = time.time()

def deferred_acceptance(proposing, not_proposing):
    
    women_temp_max_choices = [-1 for i in range(len(not_proposing))]
    
    iters = 0
    markers = [1 for i in range(len(not_proposing))] 
    # algorithm ends if the last len(not_proposing) rounds did not produce a change in matches

    while 1 in markers:
        matches = {}

        for i in range(len(not_proposing)):
            matches[i] = []        
        markers[(iters - len(not_proposing))%len(not_proposing)] = 0
        
        men_temp_max_choices = [-1 for i in range(len(proposing))]
        
        for i in range(len(proposing)):
            temp_max_value = 0
            for j in range(len(proposing[0])):
                if proposing[i][j] > temp_max_value:
                    temp_max_value = proposing[i][j]
                    men_temp_max_choices[i] = j
            if men_temp_max_choices[i]!=-1:
                matches[men_temp_max_choices[i]].append(i)
        
        print(f"Matches at iteration {iters}: {matches}")
        
        for key, value in matches.items():
            if women_temp_max_choices[key] != -1:
                temp_value = not_proposing[key][women_temp_max_choices[key]]
            else:
                temp_value = 0
            for i in range(len(value)):
                if not_proposing[key][value[i]] == temp_value:
                    continue
                elif not_proposing[key][value[i]] > temp_value: 
                    markers[(iters - len(not_proposing))%len(not_proposing)] = 1
                    if women_temp_max_choices[key] != -1:
                        proposing[women_temp_max_choices[key]][key] = 0
                    women_temp_max_choices[key] = value[i]
                    temp_value = not_proposing[key][value[i]]
                else:
                    proposing[value[i]][key] = 0
        iters+=1
    
    print(f"{iters} Iterations")
    return women_temp_max_choices

def compare_matches(new, old, compare_men_values):
    counter = 0
    for i in range(len(compare_men_values)): 
        if new[i] == None:
            continue
        if old[i] == -1 and new[i] != -1:
            counter+=1
        elif compare_men_values[i][new[i]] > compare_men_values[i][old[i]]:
            counter+=1
        elif compare_men_values[i][new[i]] < compare_men_values[i][old[i]]:
            print("This should never be reached")
    return counter

def remove_person(awpda_men_values, awpda_women_values, wpda_matches):
    if -1 in wpda_matches:
        worst_index = wpda_matches.index(-1)
    else:
        min_value = 1
        for i in range(len(wpda_matches)):
            if awpda_men_values[i][wpda_matches[i]] < min_value:
                worst_index = i
                min_value = awpda_men_values[i][wpda_matches[i]]
    del awpda_men_values[worst_index]
    awpda_women_values =  remove_col(awpda_women_values, worst_index)
    
    return awpda_men_values, awpda_women_values, worst_index

def remove_col(arr, ith):
    itg = itemgetter(*filter((ith).__ne__, range(len(arr[0]))))
    return list(map(list, map(itg, arr)))
    
def main():
    number_men = int(input("How Many Men? "))
    number_women = int(input("How Many Women? "))
    men_values = [[random.uniform(0,1) for size in range(number_women)] for _size in range(number_men)]
    women_values = [[random.uniform(0,1) for size in range(number_men)] for _size in range(number_women)]
    print(f"Men Values \n {DataFrame(men_values)}")
    print(f"Women Values \n {DataFrame(women_values)}")
    
    # use this instance of the values for WPDA

    temp_awpda_women_values = copy.deepcopy(women_values)
    temp_awpda_men_values = copy.deepcopy(men_values)
    mpda_women_values = copy.deepcopy(women_values)
    mpda_men_values = copy.deepcopy(men_values)
    compare_men_values = copy.deepcopy(men_values) # the control for comparisons
    print("\n WPDA")
    wpda_matches = deferred_acceptance(women_values, men_values)
    print("\n MPDA")
    mpda_matches = deferred_acceptance(mpda_men_values, mpda_women_values)

    print("\n WPDA, worst man leaves")

    awpda_men_values, awpda_women_values, worst_index = remove_person(temp_awpda_men_values, temp_awpda_women_values, wpda_matches)
    awpda_matches = deferred_acceptance(awpda_women_values, awpda_men_values)

    # used to put matching in men-indexed [woman, woman, ...] format
    B = [-1 for s in range(len(men_values))]  
    for index, element in enumerate(mpda_matches):
        if element != -1:
            B[element] = index
    mpda_matches = B
    awpda_matches.insert(worst_index, None)

    proposing_difference = compare_matches(mpda_matches, wpda_matches, compare_men_values)
    subtraction_difference = compare_matches(awpda_matches, wpda_matches, compare_men_values)

    print(f"WPDA \n {wpda_matches}")
    print(f"MPDA \n {mpda_matches}")
    print(f"AWPDA \n {awpda_matches}")

    print(f"Proposing Difference \n {proposing_difference}")
    print(f"Worst-outcome Man Leaves Difference \n {subtraction_difference}")

    print("--- %s seconds ---" % (time.time() - start_time))

    
    

main()
    

How Many Men? 8
How Many Women? 8
Men Values 
           0         1         2         3         4         5         6  \
0  0.115300  0.292653  0.066653  0.713358  0.909138  0.502534  0.241872   
1  0.238000  0.880654  0.832698  0.907420  0.562832  0.837141  0.882139   
2  0.246757  0.035146  0.372032  0.541191  0.169903  0.955135  0.169791   
3  0.998480  0.195818  0.700903  0.080950  0.562148  0.017132  0.572526   
4  0.430409  0.083759  0.067911  0.305696  0.852115  0.951736  0.610110   
5  0.254795  0.276112  0.162580  0.431317  0.193567  0.475856  0.244367   
6  0.894738  0.382264  0.223857  0.487782  0.314836  0.782249  0.752160   
7  0.273037  0.229485  0.561147  0.543063  0.715914  0.777220  0.285822   

          7  
0  0.982945  
1  0.345674  
2  0.936876  
3  0.634700  
4  0.452154  
5  0.464564  
6  0.108158  
7  0.117901  
Women Values 
           0         1         2         3         4         5         6  \
0  0.032458  0.855495  0.862585  0.349180  0.283793  0.790574